In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import mysql.connector
import re
import time

# MySQL 연결 설정
connection = mysql.connector.connect(
    host='localhost',
    user='ohgiraffers',
    password='ohgiraffers',
    database='test_db',
    charset='utf8mb4'
)

# Selenium 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
base_url = 'https://www.bysuco.com/product?num=60&page='
page_number = 1  # 페이지 번호 시작

while True:
    # 각 페이지에 접근
    driver.get(f'{base_url}{page_number}&orderBy=popular&category_id%5B%5D=2&keyword=&kind=bt')
    time.sleep(1)

    # 상품 목록에서 상세 페이지 링크 추출
    product_links = []
    elements = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/product/show"]')
    
    # 상품이 없으면 루프 종료
    if not elements:
        print("더 이상 상품이 없습니다. 수집을 종료합니다.")
        break

    for element in elements:
        href = element.get_attribute('href')
        product_links.append(href)

    # 각 상품 상세 페이지에서 텍스트 및 이미지 URL 추출
    for link in product_links:
        driver.get(link)
        time.sleep(1)

        try:
            # 페이지가 완전히 로드될 때까지 기다림
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'descWrap')))

            # 필요한 데이터 추출
            text_wrap = driver.find_element(By.CLASS_NAME, 'textWrap').text
            name = driver.find_element(By.CLASS_NAME, 'desc.ellipsisTwo').text
            brand_name = driver.find_element(By.CLASS_NAME, 'tit').text
            desc_wrap = driver.find_element(By.CLASS_NAME, 'descWrap').text
            
            # swiper-wrapper 내 load 클래스의 img src 속성 추출
            swiper_wrapper = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'swiper-wrapper'))
            )
            load_images = swiper_wrapper.find_elements(By.CLASS_NAME, 'load')

            # load 클래스의 img 요소 중 src 속성을 추출
            for img in load_images:
                image_url = img.get_attribute('src')
                print(f"이미지 URL: {image_url}")

            # 텍스트에서 각 항목 추출
            top_note = re.search(r"\[메인 노트\]\s*-\s*탑 노트:([^\n]+)", desc_wrap)
            top_note = top_note.group(1).strip() if top_note else "Unknown"

            middle_note = re.search(r"미들 노트:([^\n]+)", desc_wrap)
            middle_note = middle_note.group(1).strip() if middle_note else "Unknown"

            base_note = re.search(r"베이스 노트:([^\n]+)", desc_wrap)
            base_note = base_note.group(1).strip() if base_note else "Unknown"
            
            single_note = re.search(r"싱글 노트:([^\n]+)", desc_wrap)
            single_note = single_note.group(1).strip() if single_note else "Unknown"

            description = re.search(r"\[향 설명\]\s*-\s*([^\n]+)", desc_wrap)
            description = description.group(1).strip() if description else "Unknown"

            # MySQL 데이터 삽입
            with connection.cursor() as cursor:
                # 테이블 생성
                cursor.execute('''CREATE TABLE IF NOT EXISTS perfume (
                    id BIGINT AUTO_INCREMENT PRIMARY KEY,
                    name VARCHAR(50),
                    brand VARCHAR(50),
                    description TEXT
                )''')
                
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS perfume_image (
                    id BIGINT AUTO_INCREMENT PRIMARY KEY,
                    img_url TEXT,
                    perfume_id BIGINT,
                    FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                    ON DELETE CASCADE
                )
                ''')
                
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS top_note (
                    id BIGINT AUTO_INCREMENT PRIMARY KEY,
                    perfume_id BIGINT,
                    top_note TEXT,
                    FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                    ON DELETE CASCADE
                )
                ''')
                
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS middle_note (
                    id BIGINT AUTO_INCREMENT PRIMARY KEY,
                    perfume_id BIGINT,
                    middle_note TEXT,
                    FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                    ON DELETE CASCADE
                )
                ''')
                
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS base_note (
                    id BIGINT AUTO_INCREMENT PRIMARY KEY,
                    perfume_id BIGINT,
                    base_note TEXT,
                    FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                    ON DELETE CASCADE
                )
                ''')
                
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS single_note (
                    id BIGINT AUTO_INCREMENT PRIMARY KEY,
                    perfume_id BIGINT,
                    single_note TEXT,
                    FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                    ON DELETE CASCADE
                )
                ''')

                try:
                    # perfume 테이블에 데이터 삽입
                    cursor.execute("""
                        INSERT INTO perfume (name, brand, description) 
                        VALUES (%s, %s, %s)
                    """, (name, brand_name, description))
                    perfume_id = cursor.lastrowid
                    
                    # 이미지 URL을 image 테이블에 삽입
                    cursor.execute("INSERT INTO perfume_image (img_url, perfume_id) VALUES (%s,%s)", (image_url, perfume_id))
                    
                    # 각 노트 삽입
                    if top_note != "Unknown":
                        cursor.execute("INSERT INTO top_note (perfume_id, top_note) VALUES (%s, %s)", 
                                    (perfume_id, top_note))
                    if middle_note != "Unknown":
                        cursor.execute("INSERT INTO middle_note (perfume_id, middle_note) VALUES (%s, %s)", 
                                    (perfume_id, middle_note))
                    if base_note != "Unknown":
                        cursor.execute("INSERT INTO base_note (perfume_id, base_note) VALUES (%s, %s)", 
                                    (perfume_id, base_note))
                    if single_note != "Unknown":
                        cursor.execute("INSERT INTO single_note (perfume_id, single_note) VALUES (%s, %s)", 
                                    (perfume_id, single_note))
                    
                    # 모든 쿼리가 성공적으로 실행되면 커밋
                    connection.commit()

                except Exception as e:
                    # 오류 발생 시 롤백
                    connection.rollback()
                    print(f"데이터 삽입 중 오류 발생: {e}")
                    continue

        except Exception as e:
            print(f"상세 페이지에서 정보를 추출할 수 없습니다. 오류: {e}")
            continue  # 에러가 발생하면 다음 상품으로 넘어감

    # 다음 페이지로 이동
    page_number += 1

# 드라이버 종료 및 데이터베이스 연결 종료
driver.quit()
connection.close()

print("모든 데이터가 성공적으로 저장되었습니다!")

이미지 URL: https://cf.bysuco.net/4b722975836a2e17226bfc5af864e36a_cvt.webp?w=600
이미지 URL: https://cf.bysuco.net/25aa36916334ba38fde137e024b1727f_cvt.webp?w=600
이미지 URL: https://cf.bysuco.net/fc69ecc781ed188ef6af51d538fcd30d.jpg?w=600
이미지 URL: https://cf.bysuco.net/88b3a4c07b22bb85cd51c27271960828_cvt.webp?w=600
이미지 URL: https://cf.bysuco.net/0094808703aeab90541677763a69695b.jpg?w=600
이미지 URL: https://cf.bysuco.net/766a10f6d9ae4c16aea2bd3684cc164e.jpg?w=600
이미지 URL: https://cf.bysuco.net/7a0245b5fa94add4f3c26de364e3fb38.jpg?w=600
이미지 URL: https://cf.bysuco.net/11dda84bbaee1bb6d683d5ad2f9ed6d0.jpg?w=600
이미지 URL: https://cf.bysuco.net/a8cddb817ccbc59564697d04737b41d9.jpg?w=600
이미지 URL: https://cf.bysuco.net/96b9a7850d424f39f6ae155e333ff297.jpg?w=600
이미지 URL: https://cf.bysuco.net/deafe2d842ab5e093c2720f8e749bd0c.jpg?w=600
이미지 URL: https://cf.bysuco.net/5a1901d28f30a0113cac333d792b1d05.jpg?w=600
이미지 URL: https://cf.bysuco.net/38d01934a11b8b4a5c60965d585ab457.jpg?w=600
이미지 URL: https://cf.bys

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.71)
Stacktrace:
	GetHandleVerifier [0x00BF33E3+25059]
	(No symbol) [0x00B7CDE4]
	(No symbol) [0x00A5BEC3]
	(No symbol) [0x00A3D93B]
	(No symbol) [0x00AC800F]
	(No symbol) [0x00ADAE49]
	(No symbol) [0x00AC1C96]
	(No symbol) [0x00A93FAC]
	(No symbol) [0x00A94F3D]
	GetHandleVerifier [0x00EE5543+3113795]
	GetHandleVerifier [0x00EFA20A+3198986]
	GetHandleVerifier [0x00EF29E2+3168226]
	GetHandleVerifier [0x00C93250+680016]
	(No symbol) [0x00B8572D]
	(No symbol) [0x00B829D8]
	(No symbol) [0x00B82B75]
	(No symbol) [0x00B757D0]
	BaseThreadInitThunk [0x7502FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7719809E+286]
	RtlGetAppContainerNamedObjectPath [0x7719806E+238]


In [ ]:
# 향료 db만들기(image_id 매핑)
import requests
from bs4 import BeautifulSoup
import mysql.connector
import time
from mysql.connector import Error

# DB 연결
try:
    connection = mysql.connector.connect(
        host='localhost',
        user='ohgiraffers',
        password='ohgiraffers', 
        database='test_db',
        charset='utf8mb4',
        use_unicode=True
    )
    
    cursor = connection.cursor(buffered=True)
    
    # 모든 테이블 먼저 생성
    # image 테이블 생성 
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS image (
            id LONG AUTO_INCREMENT PRIMARY KEY,
            img_url TEXT NOT NULL
        )
    ''')
    
    # line 테이블 생성
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS line (
            id LONG PRIMARY KEY AUTO_INCREMENT,
            name VARCHAR(255),
            content VARCHAR(255),
            color VARCHAR(50)
        )
    ''')
    
    # spice 테이블 생성
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS spice (
            id LONG AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            content TEXT,
            img_id LONG,
            line_id LONG,
            FOREIGN KEY (img_id) REFERENCES image(id),
            FOREIGN KEY (line_id) REFERENCES line(id)
        )
    ''')

    # 테이블 생성 후 line 데이터 삽입
    cursor.execute('''
        INSERT INTO line (name, content, color) VALUES
            ('Spicy', '자극적이며 섹시한 향', 'FF5757'),
            ('Cypre', '격조있고 성숙한 느낌의 개성있는 향', 'FF7F43'),
            ('Fruity', '달콤하고 귀여운 향', 'FFBD43'),
            ('Citrus', '과일 같은 상큼한 향', 'FFE043'),
            ('Green', '상쾌하고 신선한 향', '62D66A'),
            ('Aldehyde', '인공 비누향, 모던하고 우아하면 지적인 향', '98D1FF'),
            ('Aquatic', '들과 바다 시원함이 가득한 향', '56D2FF'),
            ('Fougere', '부드럽고 감미로우며 편안한 향', 'FFD9A6'),
            ('Gourmand', '달달하고 유혹적인 미각의 즐거움을 느낄 수 있는 향', 'A1522C'),
            ('Woody', '고상하고 증후한 느낌의 따뜻하고 부드러운 향', '86390F'),
            ('Oriental', '무거우면서 부드럽고 고혹적인 향, 증후하고 달콤하며 센슈얼한 향', 'C061FF'),
            ('Floral', '화려하고 여성스러운 향', 'FF80C1'),
            ('Musk', '이성에게 어필 가능한 아름다운 향', 'F8E4FF'),
            ('Powdery', '포근한 향', 'FFFFFF'),
            ('Amber', '달콤하고 부드러운 향', 'FFA75A'),
            ('Tobacco Leather', '마초적인 남성의 개성이 강한 매력적인 향', '000000')
    ''')
    
    connection.commit()  # line 데이터 삽입 후 커밋

    url = "https://www.fragrantica.com/notes/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    notes = soup.find_all('div', class_='cell small-6 medium-4 large-3 text-center notebox')

    if not notes:
        print("향료 데이터를 찾지 못했습니다.")
    else:
        for note in notes:
            try:
                a_tag = note.find('a')
                if not a_tag:
                    continue
                    
                name = a_tag.text.strip()
                note_url = a_tag.get('href')
                img_tag = note.find('img')
                image_url = img_tag.get('src') if img_tag else None

                if not image_url:
                    continue

                # 이미지 데이터 삽입
                cursor.execute("INSERT INTO image (img_url) VALUES (%s)", (image_url,))

                additional_info = "No additional info"
                if note_url:
                    full_note_url = f"https://www.fragrantica.com{note_url}" if not note_url.startswith('http') else note_url
                    note_response = requests.get(full_note_url, headers=headers)
                    if note_response.status_code == 200:
                        note_soup = BeautifulSoup(note_response.text, 'html.parser')
                        callout_div = note_soup.find('div', class_='cell callout')
                        if callout_div:
                            additional_info = callout_div.text.strip()
                            # "Odor profile:" 제거
                            additional_info = additional_info.replace('Odor profile:', '').strip()
                    time.sleep(1)

                # spice 데이터 삽입 (line_id 제외)
                cursor.execute(
                    "INSERT INTO spice (name, content) VALUES (%s, %s)",
                    (name, additional_info)
                )
                print(f"데이터 삽입 성공 - Name: {name}")
                
                connection.commit()  # 각 항목마다 커밋
                
            except Exception as e:
                print(f"항목 처리 중 오류 발생: {e}")
                connection.rollback()
                continue

        print("모든 데이터가 성공적으로 저장되었습니다!")

except Exception as e:
    print(f"프로그램 실행 중 오류 발생: {e}")
    if connection:
        connection.rollback()

finally:
    if connection and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL 연결이 종료되었습니다.")